In [1]:
import xarray as xr
import glob
import geopandas as gpd
import sys
from   easymore import Easymore
import pandas as pd
import pint_xarray

In [4]:
# inputs
path_org = '/home/shg096/scratch/Bow_at_Calgary/'
riv = gpd.read_file(path_org+'domain/Bow_at_Calgary_riv.shp')
cat = gpd.read_file(path_org+'domain/Bow_at_Calgary_cat.shp')

In [4]:
def create_network_topology(riv,
                            cat,
                            source = 'merit'): # can be hdma, or tdx
    
    
    
    if source == 'merit':
        
        # sort by COMID
        riv.sort_values(by='COMID').reset_index(drop=True)
        cat.sort_values(by='COMID').reset_index(drop=True)
        
        # check if the riv and cat IDs are similar
        if sum(riv['COMID'].values - cat['COMID'].values) != 0:
            sys.exit('The COMID of riv and cat should be the same')
        
        # rename lengthkm to length
        riv = riv.rename(columns={'lengthkm': 'length'})
        
        # rename unitarea to area
        cat = cat.rename(columns={'unitarea': 'area'})
                         
        # centroid of cat
        cat['latitude'] = cat.centroid.y
        cat['longitude'] = cat.centroid.x
        
        # drop geometry to make gdf to df
        riv = riv.drop(columns = 'geometry')
        cat = cat.drop(columns = 'geometry')
        
        # append two datasets horizontally on COMID
        ntopo = pd.merge(riv, cat, on='COMID')
        
        # convert index to n
        ntopo = ntopo.rename_axis('n')
        
        # to xarray object,
        ntopo = ntopo.to_xarray()
        
        # add the units to the variables using pint
        ntopo = ntopo.pint.quantify({'length': 'km',
                                     'area': 'km**2',
                                     'uparea': 'km**2'})
        
        # convert dictionary
        convert = {'length': 'm',
                   'area': 'm**2',
                   'uparea': 'm**2'}
        
        # covert the units in ntopo
        ntopo = ntopo.pint.to(convert)
        ntopo = ntopo.pint.dequantify()
        
        # return the ntopo xarray object
        return ntopo
        
ntopo = create_network_topology(riv,
                            cat)

ntopo

/tmp/ipykernel_58175/3427631258.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cat['latitude'] = cat.centroid.y
/tmp/ipykernel_58175/3427631258.py:30: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cat['longitude'] = cat.centroid.x


<xarray.Dataset>
Dimensions:     (n: 453)
Coordinates:
  * n           (n) int64 0 1 2 3 4 5 6 7 8 ... 445 446 447 448 449 450 451 452
Data variables: (12/18)
    COMID       (n) int64 71022153 71022160 71022164 ... 71039250 71039266
    length      (n) float64 9.651e+03 3.002e+04 ... 2.518e+03 1.198e+04
    lengthdir   (n) float64 6.073 23.71 3.242 1.22 ... 4.938 4.428 2.041 10.24
    sinuosity   (n) float64 1.589 1.266 1.27 1.134 ... 1.277 1.25 1.234 1.17
    slope       (n) float64 0.0009722 0.0008482 0.0009692 ... 0.03278 0.005844
    uparea      (n) float64 1.172e+10 1.142e+10 ... 2.796e+07 1.137e+08
    ...          ...
    up2         (n) int64 71023606 71022426 71024578 71022444 ... 0 0 0 0
    up3         (n) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    up4         (n) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    area        (n) float64 4.133e+07 1.358e+08 1.1e+07 ... 2.796e+07 1.137e+08
    latitude    (n) float64 52.28 52.15 52.07 52.06 ... 50.93 50.92 50.95 50.94
    longitude   (n) float64 -113.9 -114.0 -114.0 -114.1 ... -115.2 -115.0 -115.3